# Scraping `wiki.epfl.ch/esn`

To login to Tequila, the `tequila-sessions` package must be installed from https://github.com/antoinealb/python-tequila

In [ ]:
import os, re, tequila
from bs4 import BeautifulSoup as bs
from tqdm import tqdm

In [ ]:
# Connect to tequila
username = ""
password = ""

conn = tequila.create_tequila_session(username, password)
conn

In [ ]:
# Get the sitemap
response = conn.get("http://wiki.epfl.ch/esn/sitemap")

# Extract the relevant pagenames
pagenames = [t.split('"')[0] for t in response.text.split("wiki.epfl.ch/esn/")][1:-2]
pagenames[:5]

In [ ]:
not_fetched = []
do_txt, do_img = True, True

for pn in tqdm(pagenames):
    raw = bs(conn.get("http://wiki.epfl.ch/esn/" + pn).text).find("div", {"id" : "content"})
    pn_new = re.sub("[^0-9a-zA-Z]+", "_", pn)
    
    # Process text
    if do_txt:
        with open(os.path.join("data", "wiki", pn_new + ".txt"), "w", encoding="utf8") as f:
            f.write(raw.get_text())
    
    # Process images
    if do_img:
        imgs = set(tag['src'] for tag in raw.findAll('img'))

        for i in imgs:
            if len(i) < 100:
                name = re.sub("[^0-9a-zA-Z.]+", "_", i.rsplit("/", 1)[1])

                try:
                    with open(os.path.join("data", "wiki", pn_new + "_" + name), "wb") as f:
                        for chunk in conn.get(i).iter_content(1024):
                            f.write(chunk)
                except:
                    not_fetched += [i,]
            else:
                not_fetched += [i,]

In [ ]:
# Lists all images that were unfetchable
for nf in not_fetched:
    if len(nf) < 1000:
        print(nf)
    else:
        print("Too long")